In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import pickle
from sklearn import metrics, tree, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,cross_val_score,train_test_split,LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

from data_load import *
from dictionaries import *

# Define the Model Parameters

model_naming_convention = model-type_encoding_directory_datastructure_column-set_version


example: nb_le_f0_d0_b0_c0_v0 is a  model which is label encoded using data set 1, column set 1, version 1 on the basic untreated data set with no data treatment for a family of multi-label classifiers

model_type will be appended to the front of the model name as it is run through each of the 

Link to model building log: https://docs.google.com/spreadsheets/d/1py4RVZ0er_JDeJo-oxY29QT6__EWHIeU6zBgp-q8Wog/edit?usp=sharing

In [21]:
directory = 'data/quiz_data.csv'
#model_name = 'model-type_encoding_directory_datastructure_column-set_version'
experiment_model_name = 'test'

column_list = [
                'problem_type', 
                'creative', 
                'outdoors', 
                'career',
                'group_work', 
                'liked_courses', 
                'disliked_courses', 
                'programming',
                'join_clubs', 
                'not_clubs', 
                'liked_projects',
                'disliked_projects',
                'tv_shows', 
                'alternate_degree', 
                'expensive_equipment', 
                'drawing',
                'essay', 
                'architecture', 
                'automotive', 
                'business', 
                'construction',
                'health',
                'environment', 
                'manufacturing', 
                'technology',
                'program'
                ]

# data_balance = {
#                 'mech': 10,
#                 'bmed': 10,
#                 'swe': 10,
#                 'tron': 10,
#                 'cive': 10,
#                 'chem': 10,
#                 'syde': 10,
#                 'msci': 10,
#                 'ce': 10,
#                 'elec': 10,
#                 'nano': 10,
#                 'geo': 10,
#                 'env': 10,
#                 'arch-e': 10,
#                 'arch': 10
#                 }
data_balance = False # this is only relevant when we want to use untreated data for code d0

data_balance_multiple = 1 # Ratio of other programs to program in binary classifier. 2 means double of other programs, 0.5 means half

test_vector = [0] * (len(column_list)-1)
test_vector = np.array(test_vector).reshape(1, -1)

### Define Encoding

 For each new type of encoding defined (other than the default label encoding) we need to define a new list of variables which are to be one hot encoded. This list name should match the encoding code that you will place in the dictionary in the model building google sheet.

 For each new type of encoding created, a new code block needs to be added under each model under each classfier family. Then, copy the code for the one hot encoded models and change the one_hot_encode list to the new list you created for this type of encoding. Once all the code blocks are added, you can run those cells!

In [22]:
ohe =  [
        'problem_type', 
        'creative', 
        'outdoors', 
        'career',
        'group_work', 
        'liked_courses', 
        'disliked_courses', 
        'programming',
        'join_clubs', 
        'not_clubs', 
        'liked_projects',
        'disliked_projects',
        'tv_shows', 
        'alternate_degree', 
        'expensive_equipment', 
        'drawing',
        'essay'
        ]

m0 =  [
        'problem_type', 
        'creative', 
        'outdoors', 
        'career',
        'group_work', 
        'liked_courses', 
        'disliked_courses', 
        'programming',
        'join_clubs', 
        'not_clubs', 
        'liked_projects',
        'disliked_projects',
        'tv_shows', 
        'alternate_degree', 
        'expensive_equipment', 
        'drawing',
        'essay'
        ]


m1 =  [
        'problem_type', 
        'creative', 
        'outdoors', 
        'career',
        'group_work', 
        'liked_courses', 
        'disliked_courses', 
        'programming',
        'join_clubs', 
        'not_clubs', 
        'liked_projects',
        'disliked_projects',
        'tv_shows', 
        'alternate_degree', 
        'expensive_equipment', 
        'drawing',
        'essay'
        ] 

## Multilabel Classifiers

### Naive Bayes - Label Encoded

In [23]:
model_name = 'nb_le_f0_'+ experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]
data = data[column_list]

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

mnb = MultinomialNB()
model = mnb.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

print(INV_INDEX_PROGRAM[model.predict(test_vector)[0]])

save_model(model,cat,model_name)
test_model(model_name,test_vector)

tron
Loading CAT file...
Loading model...
Results:
{'arch': 0.0419, 'arch-e': 0.034, 'bmed': 0.0672, 'ce': 0.1041, 'chem': 0.0629, 'cive': 0.0658, 'elec': 0.0521, 'env': 0.0521, 'geo': 0.0188, 'mech': 0.1142, 'msci': 0.0853, 'nano': 0.0651, 'swe': 0.0564, 'syde': 0.0607, 'tron': 0.1193}


### Naive Bayes - One Hot Encoded

In [24]:
model_name = 'nb_ohe_f0_'+ experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

mnb = MultinomialNB()
model = mnb.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

save_model(model,cat,model_name)

### Logistic Regression - Label Encoded

In [25]:
model_name = 'lrr_le_f0_'+ experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

LRR = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
model = LRR.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

print(INV_INDEX_PROGRAM[model.predict(test_vector)[0]])

save_model(model,cat,model_name)
test_model(model_name,test_vector)

msci
Loading CAT file...
Loading model...
Results:
{'arch': 0.069, 'arch-e': 0.0499, 'bmed': 0.0445, 'ce': 0.0454, 'chem': 0.1041, 'cive': 0.1137, 'elec': 0.0358, 'env': 0.0511, 'geo': 0.0448, 'mech': 0.0716, 'msci': 0.1308, 'nano': 0.0908, 'swe': 0.0784, 'syde': 0.0372, 'tron': 0.0331}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### Logistic Regression - One Hot Encoded

In [26]:
model_name = 'lrr_ohe_f0_'+ experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

LRR = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
model = LRR.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

save_model(model,cat,model_name)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### Support Vector Machine - Label Encoded

In [27]:
model_name = 'svm_le_f0_'+ experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

SVM = svm.SVC(probability=True)
model = SVM.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

print(INV_INDEX_PROGRAM[model.predict(test_vector)[0]])

save_model(model,cat,model_name)
test_model(model_name,test_vector)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


mech
Loading CAT file...
Loading model...
Results:
{'arch': 0.0346, 'arch-e': 0.0341, 'bmed': 0.0835, 'ce': 0.0599, 'chem': 0.0607, 'cive': 0.0887, 'elec': 0.0589, 'env': 0.0271, 'geo': 0.0206, 'mech': 0.169, 'msci': 0.0786, 'nano': 0.0827, 'swe': 0.0219, 'syde': 0.0556, 'tron': 0.124}


### Support Vector Machine - One Hot Encoded

In [28]:
model_name = 'svm_ohe_f0_'+ experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

SVM = svm.SVC(probability=True)
model = SVM.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

save_model(model,cat,model_name)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Binary Classifiers

### Naive Bayes -  Label Encoded

In [29]:
model_name = 'nb_le_f1_'+experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]
mnb = model_type = MultinomialNB()
binary_classifier(data,model_name,data_balance_multiple,mnb)

nb_le_f1_test_mech created..
nb_le_f1_test_bmed created..
nb_le_f1_test_swe created..
nb_le_f1_test_ce created..
nb_le_f1_test_tron created..
nb_le_f1_test_cive created..
nb_le_f1_test_chem created..
nb_le_f1_test_syde created..
nb_le_f1_test_msci created..
nb_le_f1_test_elec created..
nb_le_f1_test_nano created..
nb_le_f1_test_geo created..
nb_le_f1_test_env created..
nb_le_f1_test_arch-e created..
nb_le_f1_test_arch created..


### Naive Bayes - One Hot Encoded

In [30]:
model_name = 'nb_ohe_f1_'+experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)
mnb = model_type = MultinomialNB()
binary_classifier(data,model_name,data_balance_multiple,mnb)

nb_ohe_f1_test_mech created..
nb_ohe_f1_test_bmed created..
nb_ohe_f1_test_swe created..
nb_ohe_f1_test_ce created..
nb_ohe_f1_test_tron created..
nb_ohe_f1_test_cive created..
nb_ohe_f1_test_chem created..
nb_ohe_f1_test_syde created..
nb_ohe_f1_test_msci created..
nb_ohe_f1_test_elec created..
nb_ohe_f1_test_nano created..
nb_ohe_f1_test_geo created..
nb_ohe_f1_test_env created..
nb_ohe_f1_test_arch-e created..
nb_ohe_f1_test_arch created..


### Logistic Regression - Label Encoded

In [31]:
model_name = 'lrr_le_f1_'+experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]
LRR = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
binary_classifier(data,model_name,data_balance_multiple,LRR)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/li

lrr_le_f1_test_mech created..
lrr_le_f1_test_bmed created..
lrr_le_f1_test_swe created..
lrr_le_f1_test_ce created..
lrr_le_f1_test_tron created..
lrr_le_f1_test_cive created..
lrr_le_f1_test_chem created..


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/li

lrr_le_f1_test_syde created..
lrr_le_f1_test_msci created..
lrr_le_f1_test_elec created..
lrr_le_f1_test_nano created..
lrr_le_f1_test_geo created..
lrr_le_f1_test_env created..
lrr_le_f1_test_arch-e created..
lrr_le_f1_test_arch created..


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### Logistic Regression - One Hot Encoded

In [32]:
model_name = 'lrr_ohe_f1_'+experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)
LRR = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
binary_classifier(data,model_name,data_balance_multiple,LRR)

lrr_ohe_f1_test_mech created..
lrr_ohe_f1_test_bmed created..
lrr_ohe_f1_test_swe created..
lrr_ohe_f1_test_ce created..
lrr_ohe_f1_test_tron created..
lrr_ohe_f1_test_cive created..
lrr_ohe_f1_test_chem created..
lrr_ohe_f1_test_syde created..
lrr_ohe_f1_test_msci created..
lrr_ohe_f1_test_elec created..
lrr_ohe_f1_test_nano created..
lrr_ohe_f1_test_geo created..
lrr_ohe_f1_test_env created..
lrr_ohe_f1_test_arch-e created..
lrr_ohe_f1_test_arch created..


### Support Vector Machine - Label Encoded

In [33]:
model_name = 'svm_le_f1_'+experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]
SVM = svm.SVC(probability=True)
binary_classifier(data,model_name,data_balance_multiple,SVM)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

svm_le_f1_test_mech created..
svm_le_f1_test_bmed created..
svm_le_f1_test_swe created..
svm_le_f1_test_ce created..
svm_le_f1_test_tron created..
svm_le_f1_test_cive created..
svm_le_f1_test_chem created..
svm_le_f1_test_syde created..
svm_le_f1_test_msci created..
svm_le_f1_test_elec created..
svm_le_f1_test_nano created..
svm_le_f1_test_geo created..
svm_le_f1_test_env created..
svm_le_f1_test_arch-e created..
svm_le_f1_test_arch created..


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

### Support Vector Machine - One Hot Encoded

In [34]:
model_name = 'svm_ohe_f1_'+experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)
SVM = svm.SVC(probability=True)
binary_classifier(data,model_name,data_balance_multiple,SVM)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

svm_ohe_f1_test_mech created..
svm_ohe_f1_test_bmed created..
svm_ohe_f1_test_swe created..
svm_ohe_f1_test_ce created..
svm_ohe_f1_test_tron created..
svm_ohe_f1_test_cive created..
svm_ohe_f1_test_chem created..
svm_ohe_f1_test_syde created..
svm_ohe_f1_test_msci created..
svm_ohe_f1_test_elec created..


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

svm_ohe_f1_test_nano created..
svm_ohe_f1_test_geo created..
svm_ohe_f1_test_env created..
svm_ohe_f1_test_arch-e created..
svm_ohe_f1_test_arch created..


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

### Decision Tree -  Label Encoded

In [35]:
model_name = 'tree_le_f1_'+experiment_model_name
data = get_label_encoded_data(directory,model_name,column_list,'H',data_balance=data_balance)[0]
ent = tree.DecisionTreeClassifier()
binary_classifier(data,model_name,data_balance_multiple,ent)

tree_le_f1_test_mech created..
tree_le_f1_test_bmed created..
tree_le_f1_test_swe created..
tree_le_f1_test_ce created..
tree_le_f1_test_tron created..
tree_le_f1_test_cive created..
tree_le_f1_test_chem created..
tree_le_f1_test_syde created..
tree_le_f1_test_msci created..
tree_le_f1_test_elec created..
tree_le_f1_test_nano created..
tree_le_f1_test_geo created..
tree_le_f1_test_env created..
tree_le_f1_test_arch-e created..
tree_le_f1_test_arch created..


### Decision Tree - One Hot Encoded

In [36]:
model_name = 'tree_ohe_f1_'+experiment_model_name
data = get_merged_encoded_data(directory,model_name,one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=data_balance)
ent = tree.DecisionTreeClassifier()
binary_classifier(data,model_name,data_balance_multiple,ent)

tree_ohe_f1_test_mech created..
tree_ohe_f1_test_bmed created..
tree_ohe_f1_test_swe created..
tree_ohe_f1_test_ce created..
tree_ohe_f1_test_tron created..
tree_ohe_f1_test_cive created..
tree_ohe_f1_test_chem created..
tree_ohe_f1_test_syde created..
tree_ohe_f1_test_msci created..
tree_ohe_f1_test_elec created..
tree_ohe_f1_test_nano created..
tree_ohe_f1_test_geo created..
tree_ohe_f1_test_env created..
tree_ohe_f1_test_arch-e created..
tree_ohe_f1_test_arch created..


# Delete Later

In [37]:
directory = 'data/quiz_data.csv'
drop_not_happy = 'H'
data_balance

False

In [38]:
df = get_clean_data(directory,drop_not_happy)

In [39]:
programs = list(READ_PROGRAMS.values())
b_df = df.copy()
b_df = b_df.sample(n=0)

for program in programs:
    temp_df = df.copy()[df.program==program]
    while len(temp_df) <= data_balance[program]:
        temp_df = temp_df.append(temp_df)
    temp_df = temp_df.sample(n=data_balance[program])
    b_df = b_df.append(temp_df)
    b_df = b_df.sample(frac=1).reset_index(drop=True)

TypeError: 'bool' object is not subscriptable

In [21]:
print(b_df)

    program problem_type           creative  outdoors        career  \
0    arch-e      defined  somewhat_creative  outdoors     buildings   
1      nano  investigate  somewhat_creative   limited     molecules   
2        ce  investigate  somewhat_creative   limited   programming   
3      mech  investigate  somewhat_creative  outdoors    optimizing   
4        ce  investigate       not_creative   limited   programming   
5      mech  investigate  somewhat_creative  outdoors  moving_parts   
6      syde  investigate  somewhat_creative   indoors   programming   
7       env      defined  somewhat_creative   limited     resources   
8    arch-e  investigate           creative   limited     buildings   
9      msci  investigate  somewhat_creative   limited    optimizing   
10     cive      defined           creative   indoors     buildings   
11      swe  investigate  somewhat_creative   limited   programming   
12     cive      defined  somewhat_creative  outdoors     resources   
13    